In [ ]:
# !pip3 install sklearn
# !pip3 install pandas
# !pip3 install openpyxl

In [1]:
from sklearn import preprocessing
import numpy as np
import pandas as pd
import datetime

In [14]:
df = pd.read_excel('./Fluxo_Escolar_por_Escola.xlsx')
df.head(20)

,Ano,NM_DIRETORIA,NM_MUNICIPIO,CD_REDE_ENSINO,CD_ESCOLA,NM_COMPLETO_ESCOLA,APR_1,REP_1,ABA_1,APR_2,REP_2,ABA_2,APR_3,REP_3,ABA_3
0,2013,NORTE 1,SAO PAULO,1.0,12,AYRES DE MOURA PROFESSOR,0.000000e+00,0,0,9842342377,45045045,112612617,NaN,NaN,NaN
1,2013,NORTE 1,SAO PAULO,1.0,24,GAVIAO PEIXOTO BRIGADEIRO,8.781726e+09,0,1218274117,8477157593,1228426361,2944162369,6362740326,3449975204,1872843742
2,2013,NORTE 1,SAO PAULO,1.0,36,JOAO SOLIMEO,0.000000e+00,0,0,9158576202,7038835049,1375404477,6689240265,2106847191,1203912735
3,2013,NORTE 1,SAO PAULO,1.0,48,WALFREDO ARANTES CALDAS PROFESSOR,0.000000e+00,0,0,8504463959,1316964245,1785714269,743680191,2069371033,4938271523
4,2013,NORTE 1,SAO PAULO,1.0,61,SUZANA DE CAMPOS DONA,9.788136e+09,2012711763,105932206,0,0,0,NaN,NaN,NaN
5,2013,NORTE 1,SAO PAULO,1.0,73,JULIO DE FARIA E SOUZA PROFESSOR,9.874739e+09,208768263,1043841362,0,0,0,NaN,NaN,NaN
6,2013,NORTE 1,SAO PAULO,1.0,85,JULIO CESAR DE OLIVEIRA PROFESSOR,9.635316e+09,2687140226,959692895,0,0,0,NaN,NaN,NaN
7,2013,NORTE 1,SAO PAULO,1.0,97,RENATO DE ARRUDA PENTEADO PROFESSOR,0.000000e+00,0,0,8976807404,545702577,9686221123,8795309448,1172707915,1087420082
8,2013,NORTE 1,SAO PAULO,1.0,103,JOAO BOEMER JARDIM PROFESSOR,9.765854e+09,2048780441,292682916,0,0,0,NaN,NaN,NaN
9,2013,NORTE 1,SAO PAULO,1.0,115,FRANCISCO FARIA NETO PROFESSOR,9.539951e+09,1089588404,3510895967,0,0,0,NaN,NaN,NaN


In [15]:
drop_columns = ['NM_DIRETORIA', 'APR_1', 'REP_1', 'ABA_1', 'APR_2', 'REP_2', 'ABA_2']
df_cleaned = df.drop(drop_columns, axis='columns')
df_cleaned.head(20)

,Ano,NM_MUNICIPIO,CD_REDE_ENSINO,CD_ESCOLA,NM_COMPLETO_ESCOLA,APR_3,REP_3,ABA_3
0,2013,SAO PAULO,1.0,12,AYRES DE MOURA PROFESSOR,NaN,NaN,NaN
1,2013,SAO PAULO,1.0,24,GAVIAO PEIXOTO BRIGADEIRO,6362740326,3449975204,1872843742
2,2013,SAO PAULO,1.0,36,JOAO SOLIMEO,6689240265,2106847191,1203912735
3,2013,SAO PAULO,1.0,48,WALFREDO ARANTES CALDAS PROFESSOR,743680191,2069371033,4938271523
4,2013,SAO PAULO,1.0,61,SUZANA DE CAMPOS DONA,NaN,NaN,NaN
5,2013,SAO PAULO,1.0,73,JULIO DE FARIA E SOUZA PROFESSOR,NaN,NaN,NaN
6,2013,SAO PAULO,1.0,85,JULIO CESAR DE OLIVEIRA PROFESSOR,NaN,NaN,NaN
7,2013,SAO PAULO,1.0,97,RENATO DE ARRUDA PENTEADO PROFESSOR,8795309448,1172707915,1087420082
8,2013,SAO PAULO,1.0,103,JOAO BOEMER JARDIM PROFESSOR,NaN,NaN,NaN
9,2013,SAO PAULO,1.0,115,FRANCISCO FARIA NETO PROFESSOR,NaN,NaN,NaN


In [16]:
df_cleaned = df_cleaned[df_cleaned != "-"]
df_cleaned = df_cleaned[df_cleaned != datetime.time(0, 0)]
df_cleaned = df_cleaned.dropna()
df_cleaned.head(20)

,Ano,NM_MUNICIPIO,CD_REDE_ENSINO,CD_ESCOLA,NM_COMPLETO_ESCOLA,APR_3,REP_3,ABA_3
1,2013,SAO PAULO,1.0,24,GAVIAO PEIXOTO BRIGADEIRO,6362740326,3449975204,1872843742
2,2013,SAO PAULO,1.0,36,JOAO SOLIMEO,6689240265,2106847191,1203912735
3,2013,SAO PAULO,1.0,48,WALFREDO ARANTES CALDAS PROFESSOR,743680191,2069371033,4938271523
7,2013,SAO PAULO,1.0,97,RENATO DE ARRUDA PENTEADO PROFESSOR,8795309448,1172707915,1087420082
11,2013,SAO PAULO,1.0,139,MARIANO DE OLIVEIRA PROFESSOR,6358839035,2963940239,6772207737
12,2013,SAO PAULO,1.0,140,SILVIO XAVIER ANTUNES PROFESSOR,7816593933,1528384304,6550218105
14,2013,SAO PAULO,1.0,164,PIO TELLES PEIXOTO PROFESSOR,7241379547,1503448296,1255172443
15,2013,SAO PAULO,1.0,176,ZENAIDE VILALVA DE ARAUJO PROFESSORA,8579654694,1381957817,383877158
16,2013,SAO PAULO,1.0,188,ANTONIO CANDIDO CORREA GUIMARAES FILHO PROFESSOR,8237548065,1283524895,4789271832
17,2013,SAO PAULO,1.0,197,AGENOR COUTO DE MAGALHAES DOUTOR,6623655701,331182785,645161271


In [17]:
for index, row in df_cleaned.iterrows():
    if(isinstance(row['APR_3'], datetime.datetime)):
        if row['APR_3'].month < 10:
            new_value = str(row['APR_3'].day) + ".0" + str(row['APR_3'].month)
        else:
            new_value = str(row['APR_3'].day) + "." + str(row['APR_3'].month)
        df_cleaned['APR_3'] = df_cleaned['APR_3'].replace([row['APR_3']], new_value)

    if(isinstance(row['REP_3'], datetime.datetime)):
        if row['REP_3'].month < 10:
            new_value = str(row['REP_3'].day) + ".0" + str(row['REP_3'].month)
        else:
            new_value = str(row['REP_3'].day) + "." + str(row['REP_3'].month)
        df_cleaned['REP_3'] = df_cleaned['REP_3'].replace([row['REP_3']], new_value)

    if(isinstance(row['ABA_3'], datetime.datetime)):
        if row['ABA_3'].month < 10:
            new_value = str(row['ABA_3'].day) + ".0" + str(row['ABA_3'].month)
        else:
            new_value = str(row['ABA_3'].day) + "." + str(row['ABA_3'].month)
        df_cleaned['ABA_3'] = df_cleaned['ABA_3'].replace([row['ABA_3']], new_value)

df_cleaned.head(20)


,Ano,NM_MUNICIPIO,CD_REDE_ENSINO,CD_ESCOLA,NM_COMPLETO_ESCOLA,APR_3,REP_3,ABA_3
1,2013,SAO PAULO,1.0,24,GAVIAO PEIXOTO BRIGADEIRO,6362740326,3449975204,1872843742
2,2013,SAO PAULO,1.0,36,JOAO SOLIMEO,6689240265,2106847191,1203912735
3,2013,SAO PAULO,1.0,48,WALFREDO ARANTES CALDAS PROFESSOR,743680191,2069371033,4938271523
7,2013,SAO PAULO,1.0,97,RENATO DE ARRUDA PENTEADO PROFESSOR,8795309448,1172707915,1087420082
11,2013,SAO PAULO,1.0,139,MARIANO DE OLIVEIRA PROFESSOR,6358839035,2963940239,6772207737
12,2013,SAO PAULO,1.0,140,SILVIO XAVIER ANTUNES PROFESSOR,7816593933,1528384304,6550218105
14,2013,SAO PAULO,1.0,164,PIO TELLES PEIXOTO PROFESSOR,7241379547,1503448296,1255172443
15,2013,SAO PAULO,1.0,176,ZENAIDE VILALVA DE ARAUJO PROFESSORA,8579654694,1381957817,383877158
16,2013,SAO PAULO,1.0,188,ANTONIO CANDIDO CORREA GUIMARAES FILHO PROFESSOR,8237548065,1283524895,4789271832
17,2013,SAO PAULO,1.0,197,AGENOR COUTO DE MAGALHAES DOUTOR,6623655701,331182785,645161271


In [18]:
df_cleaned['ABA_3'] = df_cleaned['ABA_3'].astype(str)
df_cleaned = df_cleaned[df_cleaned["ABA_3"].str.contains("-") == False]

df_cleaned.head(20)

,Ano,NM_MUNICIPIO,CD_REDE_ENSINO,CD_ESCOLA,NM_COMPLETO_ESCOLA,APR_3,REP_3,ABA_3
1,2013,SAO PAULO,1.0,24,GAVIAO PEIXOTO BRIGADEIRO,6362740326,3449975204,1872843742
2,2013,SAO PAULO,1.0,36,JOAO SOLIMEO,6689240265,2106847191,1203912735
3,2013,SAO PAULO,1.0,48,WALFREDO ARANTES CALDAS PROFESSOR,743680191,2069371033,4938271523
7,2013,SAO PAULO,1.0,97,RENATO DE ARRUDA PENTEADO PROFESSOR,8795309448,1172707915,1087420082
11,2013,SAO PAULO,1.0,139,MARIANO DE OLIVEIRA PROFESSOR,6358839035,2963940239,6772207737
12,2013,SAO PAULO,1.0,140,SILVIO XAVIER ANTUNES PROFESSOR,7816593933,1528384304,6550218105
14,2013,SAO PAULO,1.0,164,PIO TELLES PEIXOTO PROFESSOR,7241379547,1503448296,1255172443
15,2013,SAO PAULO,1.0,176,ZENAIDE VILALVA DE ARAUJO PROFESSORA,8579654694,1381957817,383877158
16,2013,SAO PAULO,1.0,188,ANTONIO CANDIDO CORREA GUIMARAES FILHO PROFESSOR,8237548065,1283524895,4789271832
17,2013,SAO PAULO,1.0,197,AGENOR COUTO DE MAGALHAES DOUTOR,6623655701,331182785,645161271


In [19]:
df_cleaned['APR_3'] = df_cleaned['APR_3'].astype(float)
df_cleaned['REP_3'] = df_cleaned['REP_3'].astype(float)
df_cleaned['ABA_3'] = df_cleaned['ABA_3'].astype(float)

df_cleaned = df_cleaned.reset_index()
df_cleaned.head(20)

,index,Ano,NM_MUNICIPIO,CD_REDE_ENSINO,CD_ESCOLA,NM_COMPLETO_ESCOLA,APR_3,REP_3,ABA_3
0,1,2013,SAO PAULO,1.0,24,GAVIAO PEIXOTO BRIGADEIRO,6.362740e+09,3.449975e+09,1.872844e+09
1,2,2013,SAO PAULO,1.0,36,JOAO SOLIMEO,6.689240e+09,2.106847e+09,1.203913e+09
2,3,2013,SAO PAULO,1.0,48,WALFREDO ARANTES CALDAS PROFESSOR,7.436802e+08,2.069371e+09,4.938272e+09
3,7,2013,SAO PAULO,1.0,97,RENATO DE ARRUDA PENTEADO PROFESSOR,8.795309e+09,1.172708e+09,1.087420e+09
4,11,2013,SAO PAULO,1.0,139,MARIANO DE OLIVEIRA PROFESSOR,6.358839e+09,2.963940e+09,6.772208e+09
5,12,2013,SAO PAULO,1.0,140,SILVIO XAVIER ANTUNES PROFESSOR,7.816594e+09,1.528384e+09,6.550218e+09
6,14,2013,SAO PAULO,1.0,164,PIO TELLES PEIXOTO PROFESSOR,7.241380e+09,1.503448e+09,1.255172e+09
7,15,2013,SAO PAULO,1.0,176,ZENAIDE VILALVA DE ARAUJO PROFESSORA,8.579655e+09,1.381958e+09,3.838772e+08
8,16,2013,SAO PAULO,1.0,188,ANTONIO CANDIDO CORREA GUIMARAES FILHO PROFESSOR,8.237548e+09,1.283525e+09,4.789272e+09
9,17,2013,SAO PAULO,1.0,197,AGENOR COUTO DE MAGALHAES DOUTOR,6.623656e+09,3.311828e+08,6.451613e+08


In [20]:
df_cleaned = df_cleaned.rename(columns={'Ano':'ano', 'NM_MUNICIPIO':'municipio', 'CD_REDE_ENSINO':'id_rede_ensino', 'CD_ESCOLA':'id_escola', 'NM_COMPLETO_ESCOLA':'escola', 'APR_3':'aprovado', 'REP_3':'reprovado', 'ABA_3':'abandono'})
df_cleaned.head(20)

,index,ano,municipio,id_rede_ensino,id_escola,escola,aprovado,reprovado,abandono
0,1,2013,SAO PAULO,1.0,24,GAVIAO PEIXOTO BRIGADEIRO,6.362740e+09,3.449975e+09,1.872844e+09
1,2,2013,SAO PAULO,1.0,36,JOAO SOLIMEO,6.689240e+09,2.106847e+09,1.203913e+09
2,3,2013,SAO PAULO,1.0,48,WALFREDO ARANTES CALDAS PROFESSOR,7.436802e+08,2.069371e+09,4.938272e+09
3,7,2013,SAO PAULO,1.0,97,RENATO DE ARRUDA PENTEADO PROFESSOR,8.795309e+09,1.172708e+09,1.087420e+09
4,11,2013,SAO PAULO,1.0,139,MARIANO DE OLIVEIRA PROFESSOR,6.358839e+09,2.963940e+09,6.772208e+09
5,12,2013,SAO PAULO,1.0,140,SILVIO XAVIER ANTUNES PROFESSOR,7.816594e+09,1.528384e+09,6.550218e+09
6,14,2013,SAO PAULO,1.0,164,PIO TELLES PEIXOTO PROFESSOR,7.241380e+09,1.503448e+09,1.255172e+09
7,15,2013,SAO PAULO,1.0,176,ZENAIDE VILALVA DE ARAUJO PROFESSORA,8.579655e+09,1.381958e+09,3.838772e+08
8,16,2013,SAO PAULO,1.0,188,ANTONIO CANDIDO CORREA GUIMARAES FILHO PROFESSOR,8.237548e+09,1.283525e+09,4.789272e+09
9,17,2013,SAO PAULO,1.0,197,AGENOR COUTO DE MAGALHAES DOUTOR,6.623656e+09,3.311828e+08,6.451613e+08


In [21]:
normalized_list = preprocessing.normalize([df_cleaned['aprovado'],df_cleaned['reprovado'],df_cleaned['abandono']])

df_cleaned['aprovado_normalized'] = normalized_list[0]
df_cleaned['reprovado_normalized'] = normalized_list[1]
df_cleaned['abandono_normalized'] = normalized_list[2]

df_cleaned.head(20)

,index,ano,municipio,id_rede_ensino,id_escola,escola,aprovado,reprovado,abandono,aprovado_normalized,reprovado_normalized,abandono_normalized
0,1,2013,SAO PAULO,1.0,24,GAVIAO PEIXOTO BRIGADEIRO,6.362740e+09,3.449975e+09,1.872844e+09,0.010096,1.029664e-02,0.005552
1,2,2013,SAO PAULO,1.0,36,JOAO SOLIMEO,6.689240e+09,2.106847e+09,1.203913e+09,0.010614,6.288000e-03,0.003569
2,3,2013,SAO PAULO,1.0,48,WALFREDO ARANTES CALDAS PROFESSOR,7.436802e+08,2.069371e+09,4.938272e+09,0.001180,6.176150e-03,0.014639
3,7,2013,SAO PAULO,1.0,97,RENATO DE ARRUDA PENTEADO PROFESSOR,8.795309e+09,1.172708e+09,1.087420e+09,0.013955,3.500010e-03,0.003224
4,11,2013,SAO PAULO,1.0,139,MARIANO DE OLIVEIRA PROFESSOR,6.358839e+09,2.963940e+09,6.772208e+09,0.010089,8.846040e-03,0.020076
5,12,2013,SAO PAULO,1.0,140,SILVIO XAVIER ANTUNES PROFESSOR,7.816594e+09,1.528384e+09,6.550218e+09,0.012402,4.561546e-03,0.019418
6,14,2013,SAO PAULO,1.0,164,PIO TELLES PEIXOTO PROFESSOR,7.241380e+09,1.503448e+09,1.255172e+09,0.011490,4.487123e-03,0.003721
7,15,2013,SAO PAULO,1.0,176,ZENAIDE VILALVA DE ARAUJO PROFESSORA,8.579655e+09,1.381958e+09,3.838772e+08,0.013613,4.124528e-03,0.001138
8,16,2013,SAO PAULO,1.0,188,ANTONIO CANDIDO CORREA GUIMARAES FILHO PROFESSOR,8.237548e+09,1.283525e+09,4.789272e+09,0.013070,3.830750e-03,0.014198
9,17,2013,SAO PAULO,1.0,197,AGENOR COUTO DE MAGALHAES DOUTOR,6.623656e+09,3.311828e+08,6.451613e+08,0.010510,9.884330e-04,0.001913


In [22]:
df_cleaned.to_excel('FluxoEscolarNormalizado.xlsx', encoding="utf-8")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
